<a href="https://colab.research.google.com/github/ismachy/Introduction-to-ML/blob/main/HW_5_Q_02_a_b_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
housing = pd.read_csv('/content/drive/MyDrive/dataset/Housing2.csv')
housing

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [ ]:
num_vars = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking']
df_new = housing[num_vars]

In [ ]:
scaler = MinMaxScaler()
df_new[num_vars] = scaler.fit_transform(df_new[num_vars])
df_new.head()

<ipython-input-24-e90be4113ded>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[num_vars] = scaler.fit_transform(df_new[num_vars])


,price,area,bedrooms,bathrooms,stories,parking
0,1.000000,0.396564,0.6,0.333333,0.666667,0.666667
1,0.909091,0.502405,0.6,1.000000,1.000000,1.000000
2,0.909091,0.571134,0.4,0.333333,0.333333,0.666667
3,0.906061,0.402062,0.6,0.333333,0.333333,1.000000
4,0.836364,0.396564,0.6,0.000000,0.333333,0.666667


In [ ]:
X = df_new.iloc[:, 1:6].values
Y = df_new.iloc[:, 0].values

In [ ]:
X = torch.tensor(X)
Y = torch.tensor(Y)

In [ ]:
def model(X, W1, W2, W3, W4, W5, B):
    return W5*X[:,4] + W4*X[:,3] + W3*X[:,2] + W2*X[:,1] + W1*X[:,0] + B

In [ ]:
def loss_fn(Y_p, Y):
    squared_diffs = (Y_p - Y)**2
    return squared_diffs.mean()

In [ ]:
W1 = torch.ones(())
W2 = torch.ones(())
W3 = torch.ones(())
W4 = torch.ones(())
W5 = torch.ones(())
B = torch.zeros(())

In [ ]:
n_samples = X.shape[0]
n_val = int(0.2 * n_samples)
shuffled_indices = torch.randperm(n_samples)
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [ ]:
train_X = X[train_indices]
train_Y = Y[train_indices]
val_X = X[val_indices]
val_Y = Y[val_indices]

In [ ]:
def training_loop(n_epochs, optimizer, params, train_X, val_X, train_Y, val_Y):

    for epoch in range(1, n_epochs + 1):
        train_Y_p = model(train_X, *params)
        train_loss = loss_fn(train_Y_p, train_Y)

        val_Y_p = model(val_X, *params)
        val_loss = loss_fn(val_Y_p, val_Y)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")

    return params

In [ ]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-1
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.0111, Validation loss 0.0131
Epoch 1000, Training loss 0.0111, Validation loss 0.0131
Epoch 1500, Training loss 0.0111, Validation loss 0.0131
Epoch 2000, Training loss 0.0111, Validation loss 0.0131
Epoch 2500, Training loss 0.0111, Validation loss 0.0131
Epoch 3000, Training loss 0.0111, Validation loss 0.0131
Epoch 3500, Training loss 0.0111, Validation loss 0.0131
Epoch 4000, Training loss 0.0111, Validation loss 0.0131
Epoch 4500, Training loss 0.0111, Validation loss 0.0131
Epoch 5000, Training loss 0.0111, Validation loss 0.0131


tensor([0.4616, 0.0600, 0.2944, 0.1489, 0.0863, 0.0366], requires_grad=True)

In [ ]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.0132, Validation loss 0.0151
Epoch 1000, Training loss 0.0113, Validation loss 0.0132
Epoch 1500, Training loss 0.0111, Validation loss 0.0131
Epoch 2000, Training loss 0.0111, Validation loss 0.0131
Epoch 2500, Training loss 0.0111, Validation loss 0.0131
Epoch 3000, Training loss 0.0111, Validation loss 0.0131
Epoch 3500, Training loss 0.0111, Validation loss 0.0131
Epoch 4000, Training loss 0.0111, Validation loss 0.0131
Epoch 4500, Training loss 0.0111, Validation loss 0.0131
Epoch 5000, Training loss 0.0111, Validation loss 0.0131


tensor([0.4616, 0.0600, 0.2944, 0.1489, 0.0863, 0.0366], requires_grad=True)

In [ ]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-3
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.1288, Validation loss 0.1134
Epoch 1000, Training loss 0.0508, Validation loss 0.0561
Epoch 1500, Training loss 0.0342, Validation loss 0.0391
Epoch 2000, Training loss 0.0232, Validation loss 0.0268
Epoch 2500, Training loss 0.0173, Validation loss 0.0200
Epoch 3000, Training loss 0.0146, Validation loss 0.0168
Epoch 3500, Training loss 0.0133, Validation loss 0.0153
Epoch 4000, Training loss 0.0125, Validation loss 0.0144
Epoch 4500, Training loss 0.0120, Validation loss 0.0138
Epoch 5000, Training loss 0.0115, Validation loss 0.0134


tensor([ 0.4832,  0.2128,  0.2521,  0.1265,  0.0837, -0.0196],
       requires_grad=True)

In [ ]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-4
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 1.0239, Validation loss 0.8956
Epoch 1000, Training loss 0.8240, Validation loss 0.7165
Epoch 1500, Training loss 0.6554, Validation loss 0.5664
Epoch 2000, Training loss 0.5144, Validation loss 0.4419
Epoch 2500, Training loss 0.3981, Validation loss 0.3402
Epoch 3000, Training loss 0.3038, Validation loss 0.2588
Epoch 3500, Training loss 0.2291, Validation loss 0.1955
Epoch 4000, Training loss 0.1719, Validation loss 0.1480
Epoch 4500, Training loss 0.1298, Validation loss 0.1142
Epoch 5000, Training loss 0.1003, Validation loss 0.0917


tensor([ 0.6142,  0.6137,  0.5931,  0.5954,  0.5938, -0.3722],
       requires_grad=True)

In [ ]:
# SGD

In [ ]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-1
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.0114, Validation loss 0.0135
Epoch 1000, Training loss 0.0111, Validation loss 0.0131
Epoch 1500, Training loss 0.0111, Validation loss 0.0131
Epoch 2000, Training loss 0.0111, Validation loss 0.0131
Epoch 2500, Training loss 0.0111, Validation loss 0.0131
Epoch 3000, Training loss 0.0111, Validation loss 0.0131
Epoch 3500, Training loss 0.0111, Validation loss 0.0131
Epoch 4000, Training loss 0.0111, Validation loss 0.0131
Epoch 4500, Training loss 0.0111, Validation loss 0.0131
Epoch 5000, Training loss 0.0111, Validation loss 0.0131


tensor([0.4616, 0.0600, 0.2944, 0.1489, 0.0863, 0.0366], requires_grad=True)

In [ ]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.0488, Validation loss 0.0567
Epoch 1000, Training loss 0.0222, Validation loss 0.0266
Epoch 1500, Training loss 0.0157, Validation loss 0.0188
Epoch 2000, Training loss 0.0136, Validation loss 0.0161
Epoch 2500, Training loss 0.0127, Validation loss 0.0150
Epoch 3000, Training loss 0.0122, Validation loss 0.0144
Epoch 3500, Training loss 0.0118, Validation loss 0.0140
Epoch 4000, Training loss 0.0116, Validation loss 0.0138
Epoch 4500, Training loss 0.0115, Validation loss 0.0136
Epoch 5000, Training loss 0.0114, Validation loss 0.0135


tensor([ 0.4989,  0.1716,  0.2804,  0.1237,  0.0762, -0.0063],
       requires_grad=True)

In [ ]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-3
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.2200, Validation loss 0.1995
Epoch 1000, Training loss 0.1351, Validation loss 0.1406
Epoch 1500, Training loss 0.1142, Validation loss 0.1249
Epoch 2000, Training loss 0.0997, Validation loss 0.1110
Epoch 2500, Training loss 0.0876, Validation loss 0.0985
Epoch 3000, Training loss 0.0772, Validation loss 0.0875
Epoch 3500, Training loss 0.0683, Validation loss 0.0779
Epoch 4000, Training loss 0.0607, Validation loss 0.0698
Epoch 4500, Training loss 0.0543, Validation loss 0.0627
Epoch 5000, Training loss 0.0487, Validation loss 0.0566


tensor([ 0.7130,  0.5657,  0.6546,  0.3446,  0.3684, -0.4038],
       requires_grad=True)

In [ ]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-4
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.9961, Validation loss 0.8697
Epoch 1000, Training loss 0.7953, Validation loss 0.6901
Epoch 1500, Training loss 0.6423, Validation loss 0.5548
Epoch 2000, Training loss 0.5255, Validation loss 0.4529
Epoch 2500, Training loss 0.4362, Validation loss 0.3763
Epoch 3000, Training loss 0.3679, Validation loss 0.3186
Epoch 3500, Training loss 0.3154, Validation loss 0.2752
Epoch 4000, Training loss 0.2751, Validation loss 0.2426
Epoch 4500, Training loss 0.2440, Validation loss 0.2180
Epoch 5000, Training loss 0.2198, Validation loss 0.1994


tensor([ 0.8508,  0.7553,  0.9025,  0.7705,  0.7914, -0.5143],
       requires_grad=True)